# Wikipedia Politician Articles: Bias

This notebook explores bias in data using Wikipedia articles about political figures from different countries. The analysis aims to examine the coverage of politicians on Wikipedia and the quality of articles about politicians across nations. 

The notebook utilizes data from two sources: 
- Dataset of Wikipedia articles about politicians using Wikipedia [Category:Politicians_by_nationality](https://en.wikipedia.org/wiki/Category:Politicians_by_nationality).
- Dataset of country populations obtained from [World Population Data Sheet](https://www.prb.org/international/indicator/population/table/).

Additionally, the machine learning service ORES is used to estimate the quality of each article.

## Article Page Info MediaWiki API

The following sub-section contains code to access page info data using the [MediaWiki REST API for the EN Wikipedia](https://www.mediawiki.org/wiki/API:Main_page). This sub-section shows how to request summary 'page info' for a single article page. The API documentation, [API:Info](https://www.mediawiki.org/wiki/API:Info), covers additional details that may be helpful when trying to use or understand this example.

### License

This code example was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/).

Modifications to this code were made by Himanshu Naidu on October 13, 2024.

In [1]:
# 
# These are standard python modules
import json, time, urllib.parse
#
# The 'requests' module is not a standard Python module. You will need to install this with pip/pip3 if you do not already have it
import requests

The example relies on some constants that help make the code a bit more readable.

In [2]:
#########
#
#    CONSTANTS
#

# The basic English Wikipedia API endpoint
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"
API_HEADER_AGENT = 'User-Agent'

# We'll assume that there needs to be some throttling for these requests - we should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': '<hnaidu36@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2024'
}

# This is just a list of English Wikipedia article titles that we can use for example requests
ARTICLE_TITLES = [ 'Bison', 'Northern flicker', 'Red squirrel', 'Chinook salmon', 'Horseshoe bat' ]

# This is a string of additional page properties that can be returned see the Info documentation for
# what can be included. If you don't want any this can simply be the empty string
PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"
#PAGEINFO_EXTENDED_PROPERTIES = ""

# This template lists the basic parameters for making this
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",           # to simplify this should be a single page title at a time
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}


The API request will be made using one procedure. The idea is to make this reusable. The procedure is parameterized, but relies on the constants above for the important parameters. The underlying assumption is that this will be used to request data for a set of article pages. Therefore the parameter most likely to change is the article_title.

In [3]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    
    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['titles'] = article_title

    if not request_template['titles']:
        raise Exception("Must supply an article title to make a pageinfo request.")

    if API_HEADER_AGENT not in headers:
        raise Exception(f"The header data should include a '{API_HEADER_AGENT}' field that contains your UW email address.")

    if 'uwnetid@uw' in headers[API_HEADER_AGENT]:
        raise Exception(f"Use your UW email address in the '{API_HEADER_AGENT}' field.")

    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or any other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


In [4]:
print(f"Getting page info data for: {ARTICLE_TITLES[3]}")
info = request_pageinfo_per_article(ARTICLE_TITLES[3])
print(json.dumps(info,indent=4))

Getting page info data for: Chinook salmon
{
    "batchcomplete": "",
    "query": {
        "pages": {
            "1212891": {
                "pageid": 1212891,
                "ns": 0,
                "title": "Chinook salmon",
                "contentmodel": "wikitext",
                "pagelanguage": "en",
                "pagelanguagehtmlcode": "en",
                "pagelanguagedir": "ltr",
                "touched": "2024-10-12T10:13:54Z",
                "lastrevid": 1234351318,
                "length": 53787,
                "watchers": 109,
                "talkid": 3909817,
                "fullurl": "https://en.wikipedia.org/wiki/Chinook_salmon",
                "editurl": "https://en.wikipedia.org/w/index.php?title=Chinook_salmon&action=edit",
                "canonicalurl": "https://en.wikipedia.org/wiki/Chinook_salmon"
            }
        }
    }
}


## Requesting ORES scores through LiftWing ML Service API

This sub-section illustrates how to generate article quality estimates for article revisions using the LiftWing version of ORES. The ORES API documentation can be accessed from the main ORES page. The ORES LiftWing documentation is very thin ... even thinner than the standard ORES documentation. Further, it is clear that some parameters have been renamed (e.g., "revid" in the old ORES API is now "rev_id" in the LiftWing ORES API).

### License
This code example was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). 

Modifications to this code were made by Himanshu Naidu on October 13, 2024.

In [6]:
#########
#
#    CONSTANTS
#

#    The current LiftWing ORES API endpoint and prediction model
#
API_ORES_LIFTWING_ENDPOINT = "https://api.wikimedia.org/service/lw/inference/v1/models/{model_name}:predict"
API_ORES_EN_QUALITY_MODEL = "enwiki-articlequality"

#
#    The throttling rate is a function of the Access token that you are granted when you request the token. The constants
#    come from dissecting the token and getting the rate limits from the granted token. An example of that is below.
#
API_ORES_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_ORES_THROTTLE_WAIT = ((60.0*60.0)/5000.0)-API_ORES_LATENCY_ASSUMED  # The key authorizes 5000 requests per hour

#    When making automated requests we should include something that is unique to the person making the request
#    This should include an email - your UW email would be good to put in there
#    
#    Because all LiftWing API requests require some form of authentication, you need to provide your access token
#    as part of the header too
#
REQUEST_HEADER_TEMPLATE = {
    'User-Agent': "<{email_address}>, University of Washington, MSDS DATA 512 - AUTUMN 2024",
    'Content-Type': 'application/json',
    'Authorization': "Bearer {access_token}"
}
#
#    This is a template for the parameters that we need to supply in the headers of an API request
#
REQUEST_HEADER_PARAMS_TEMPLATE = {
    'email_address' : "",         # your email address should go here
    'access_token'  : ""          # the access token you create will need to go here
}

#
#    A dictionary of English Wikipedia article titles (keys) and sample revision IDs that can be used for this ORES scoring example
#
ARTICLE_REVISIONS = { 'Bison':1085687913 , 'Northern flicker':1086582504 , 'Red squirrel':1083787665 , 'Chinook salmon':1085406228 , 'Horseshoe bat':1060601936 }

#
#    This is a template of the data required as a payload when making a scoring request of the ORES model
#
ORES_REQUEST_DATA_TEMPLATE = {
    "lang":        "en",     # required that its english - we're scoring English Wikipedia revisions
    "rev_id":      "",       # this request requires a revision id
    "features":    True
}

#
#    These are used later - defined here so they, at least, have empty values
#
USERNAME = ""
ACCESS_TOKEN = ""
#

### Get your access token

You will need a Wikimedia user account to get access to Lift Wing (the ML API service). You can either [create an account or login](https://api.wikimedia.org/w/index.php?title=Special:UserLogin). If you have a Wikipedia user account - you might already have an Wikimedia account. If you are not sure try your Wikipedia username and password to check it. If you do not have a Wikimedia account you will need to create an account that you can use to get an access token.

There is [a 'guide' that describes how to get authentication tokens](https://api.wikimedia.org/wiki/Authentication) - but not everything works the way it is described in that documentation. You should review that documentation and then read the rest of this comment.

The documentation talks about using a "dashboard" for managing authentication tokens. That's a rather generous description for what looks like a simple list of token things. You might have a hard time finding this "dashboard". First, on the left hand side of the page, you'll see a column of links. The bottom section is a set of links titled "Tools". In that section is a link that says [Special pages](https://api.wikimedia.org/wiki/Special:SpecialPages) which will take you to a list of ... well, special pages. At the very bottom of the "Special pages" page is a section titled "Other special pages" (scroll all the way to the bottom). The first link in that section is called [API keys](https://api.wikimedia.org/wiki/Special:AppManagement). When you get to the "API keys" page you can create a new key.

The authentication guide suggests that you should create a server-side app key. This does not seem to work correctly - as yet. It failed on multiple attempts when I attempted to create a server-side app key. BUT, there is an option to create a [Personal API token](https://api.wikimedia.org/wiki/Authentication) that should work for this course and the type of ORES page scoring that you will need to perform.

Note, when you create a Personal API token you are granted the three items - a Client ID, a Client secret, and a Access token - you shold save all three of these. When you dismiss the box they are gone. If you lose any one of the tokens you can destroy or deactivate the Personal API token from the dashboard and then create a new one.

The value you need to work the code below is the Access token - a very long string.


In [107]:
#   Once you've done the right set up with your Wikimedia account, it should provide you with three different keys, a Client ID,
#   a Client secret, and a Access token.
#
#   In this case I don't want to distribute my keys with the source of the notebook, so I used the dotenv package to load them from
#   a file called '.env' in the same directory as this notebook. The file should look like this:
#
#   WIKIMEDIA_USERNAME="<your_wikimedia_username>"
#   WIKIMEDIA_CLIENT_ID="<your_wikimedia_client_id>"
#   WIKIMEDIA_CLIENT_SECRET="<your_wikimedia_client_secret>"
#   WIKIMEDIA_ACCESS_TOKEN="<your_wikimedia_provided_access_token_its_a_really_long_string>"
#
#   The repository should have a file called '.env.example' that you can copy to '.env' and fill in the values.

# USERNAME = "<your_wikimedia_username>"
# ACCESS_TOKEN = "<your_wikimedia_provided_access_token_its_a_really_long_string>"

# Note: The above properties will be assigned in the "Getting Article Quality Predictions" section

### Define a function to make the ORES API request

The API request will be made using a function to encapsulate call and make access reusable in other notebooks. The procedure is parameterized, relying on the constants above for some important default parameters. The primary assumption is that this function will be used to request data for a set of article revisions. The main parameter is 'article_revid'. One should be able to simply pass in a new article revision id on each call and get back a python dictionary as the result. A valid result will be a dictionary that contains the probabilities that the specific revision is one of six different article quality levels. Generally, quality level with the highest probability score is considered the quality level for the article. This can be tricky when you have two (or more) highly probable quality levels.

In [7]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_ores_score_per_article(article_revid = None, email_address=None, access_token=None,
                                   endpoint_url = API_ORES_LIFTWING_ENDPOINT, 
                                   model_name = API_ORES_EN_QUALITY_MODEL, 
                                   request_data = ORES_REQUEST_DATA_TEMPLATE, 
                                   header_format = REQUEST_HEADER_TEMPLATE, 
                                   header_params = REQUEST_HEADER_PARAMS_TEMPLATE):
    
    #    Make sure we have an article revision id, email and token
    #    This approach prioritizes the parameters passed in when making the call
    if article_revid:
        request_data['rev_id'] = article_revid
    if email_address:
        header_params['email_address'] = email_address
    if access_token:
        header_params['access_token'] = access_token
    
    #   Making a request requires a revision id - an email address - and the access token
    if not request_data['rev_id']:
        raise Exception("Must provide an article revision id (rev_id) to score articles")
    if not header_params['email_address']:
        raise Exception("Must provide an 'email_address' value")
    if not header_params['access_token']:
        raise Exception("Must provide an 'access_token' value")
    
    # Create the request URL with the specified model parameter - default is a article quality score request
    request_url = endpoint_url.format(model_name=model_name)
    
    # Create a compliant request header from the template and the supplied parameters
    headers = dict()
    for key in header_format.keys():
        headers[str(key)] = header_format[key].format(**header_params)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free data
        # source like ORES - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        #response = requests.get(request_url, headers=headers)
        response = requests.post(request_url, headers=headers, data=json.dumps(request_data))
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


## Getting the Article and Population Data

In [11]:
#########
#
#    IMPORTS
#

import numpy as np
import pandas as pd

In [12]:
#########
#
#    CONSTANTS
#
# Dev environment flag to control the number of requests made to the API for testing
DEV_ENVIRONMENT = False

# The cleaned CSV file path that contains the list of politicians by country
POLITICIANS_BY_COUNTRY_CSV_PATH = "politicians_by_country_AUG.2024.csv"
# The cleaned CSV file path that contains the list of population by country
POPULATION_BY_COUNTRY_CSV_PATH = "population_by_country_AUG.2024.csv"

As stated in the [MediaWiki REST API Help](https://www.mediawiki.org/w/api.php?action=help&modules=query): 

For titles "Maximum number of values is 50 (500 for clients that are allowed higher limits)."

In [57]:
# To be on the safe side, we'll limit the number of titles to 40
TITLE_LIMIT = 40
# The separator for the titles in the query string
TITLE_SEPARATOR = "|"

In [105]:
# The path to the error list file, that includes the articles whose page info could not be retrieved
ERROR_LIST_PATH = "data/errors.txt" 
# The path to the output file that contains the page info data
WP_POLITICIANS_WITH_PAGE_INFO_PATH = "data/wp_politicians_with_page_info.csv"

There is a way to get the page information for multiple pages at the same time, by separating the page titles with the vertical bar "|" character. However, this approach has limits. You should probably check the API documentation if you want to do multiple pages in a single request - and limit the number of pages in one request reasonably.

In [90]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_page_info_multiple_articles(titles = None, title_sep = '|', error_list = None):
    '''
    This function takes a list of article titles and returns the page information for all the articles in a list.
    This function also adds the titles that could not be fetched to the error list.

    Parameters:
    ------------
    titles : str
        A string of article titles separated by a delimiter

    title_sep : str
        The delimiter that separates the article titles in the string

    error_list : list
        A list to store the titles that could not be fetched

    Returns:
    ------------
    page_info_array : list
        A list of dictionaries containing the page information for each article
    '''
    page_info_array = []
    page_info = None

    if titles is None:
        titles = ""
    if error_list is None:
        error_list = []
    try:
        json_response = request_pageinfo_per_article(titles)

        page_info_dict = json_response['query']['pages']
        for page_id, page_info in page_info_dict.items():
            try:
                page_info_array.append({
                    "page_id": page_info["pageid"],
                    "title": page_info["title"],
                    "revision_id": page_info["lastrevid"],
                    "page_length": page_info["length"],
                })
            except Exception as e:
                print("Error while fetching data for: ", page_info["title"])
                error_list.append(page_info["title"])
                print(e)
    
    except Exception as e:
        print("Error in API while fetching data")
        error_list.extend(titles.split(title_sep))
        print(e)
    
    return page_info_array

In [103]:
def write_list_to_file(file_path, list_to_write):
    '''
    This function writes a list to a file.

    Parameters:
    ------------
    file_path : str
        The path of the file to write the list to
    
    list_to_write : list
        The list to write to the file
    '''
    with open(file_path, 'w') as file:
        for item in list_to_write:
            file.write("%s\n" % item)

### Fetch Politicians Data

In [91]:
# Load the cleaned CSV file that contains the list of politicians by country
politicians_by_country_df = pd.read_csv(POLITICIANS_BY_COUNTRY_CSV_PATH)

In [92]:
page_info_array = []
error_list = []
total_len = 0

# Get the page info in batches of TITLE_LIMIT
# The function np.array_split splits the dataframe into batches of TITLE_LIMIT or TITLE_LIMIT + 1
# Hence, we use a TITLE_LIMIT that is less than the actual limit to avoid making a bigger request than the limit
for batch in np.array_split(politicians_by_country_df, len(politicians_by_country_df) // TITLE_LIMIT):
    titles = batch["name"].tolist()
    page_info_array_batch = request_page_info_multiple_articles(TITLE_SEPARATOR.join(titles), TITLE_SEPARATOR, error_list)
    page_info_array.extend(page_info_array_batch)

page_info_df = pd.DataFrame(page_info_array)

# Write the error list to a file
write_list_to_file(ERROR_LIST_PATH, error_list)

/home/hnaidu36/miniforge3/envs/data512/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Error while fetching data for:  Barbara Eibinger-Miedl
'pageid'
Error while fetching data for:  Mehrali Gasimov
'pageid'
Error while fetching data for:  Kyaw Myint
'pageid'
Error while fetching data for:  André Ngongang Ouandji
'pageid'
Error while fetching data for:  Tomás Pimentel
'pageid'
Error while fetching data for:  Richard Sumah
'pageid'
Error while fetching data for:  Segun ''Aeroland'' Adewale
'pageid'
Error while fetching data for:  Bashir Bililiqo
'pageid'


In [98]:
# Calculate Error Rate
error_rate = len(error_list) / page_info_df['page_id'].nunique()
print(f"Error Rate: {error_rate * 100}%")

Error Rate: 0.11262846684499507%


As it turns out, there are duplicates in the politicians_by_country_df dataframe. We'll drop the duplicates in the page_info_df dataframe, and keep the first occurrence. 

(Ideally, we should have dropped the duplicates in the politicians_by_country_df dataframe)

In [104]:
page_info_df = page_info_df.drop_duplicates(subset=['page_id'])
page_info_df.shape

(7103, 4)

In [106]:
# Write the final page_info dataframe to a CSV file
page_info_df.to_csv(WP_POLITICIANS_WITH_PAGE_INFO_PATH, index=False)

## Getting Article Quality Predictions

Once the latest revision ids are extracted for each article, using the Page Info API, the ORES API can be used to get the Quality Predictions

In [109]:
#########
#
#    IMPORTS
#
import os

from dotenv import load_dotenv

In [108]:
#########
#
#    CONSTANTS
#

# The path to the .env file that contains the Wikimedia credentials
ENV_PATH = ".env"

In [ ]:
#########
#
#    PROCEDURES/FUNCTIONS
#


### Loading the Environment

In [112]:
load_dotenv(ENV_PATH)

USERNAME = os.getenv("WIKIMEDIA_USERNAME")
ACCESS_TOKEN = os.getenv("WIKIMEDIA_ACCESS_TOKEN")

## Combining the Datasets

## Analysis

## Results

## Reflections and Implications

## Conclusion